In [1]:
# Importing libraries

import pandas as pd
import numpy as np
import os

In [2]:
# Importing file

ords_prods_merge = pd.read_pickle(r'C:\Users\casbacar\Documents\Study\Data Immersion\Achievement 4\03_2025_Instacart basked analysis\03 Scripts\Prepared data\ ords_hour_day_busy_4.7.pkl')

In [3]:
ords_prods_merge.head()

,product_id,product_name,aisle_id,department_id,prices,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,add_to_cart_order,reordered,_merge,Busiest days,Busiest hours
0,1,Chocolate Sandwich Cookies,61,19,5.8,3139998,138,prior,28,6,11,3.0,5,0,both,Regularly busy,Fewer orders
1,1,Chocolate Sandwich Cookies,61,19,5.8,1977647,138,prior,30,6,17,20.0,1,1,both,Regularly busy,Most orders
2,1,Chocolate Sandwich Cookies,61,19,5.8,389851,709,prior,2,0,21,6.0,20,0,both,Busiest days,Most orders
3,1,Chocolate Sandwich Cookies,61,19,5.8,652770,764,prior,1,3,13,NaN,10,0,both,Least busy days,Average orders
4,1,Chocolate Sandwich Cookies,61,19,5.8,1813452,764,prior,3,4,17,9.0,11,1,both,Least busy days,Most orders


## Step 2

In [4]:

ords_prods_merge.groupby('department_id').agg({
    'order_number': ['mean']
})

,order_number
,mean
department_id,
1,15.457838
2,17.277920
3,17.170395
4,17.811403
5,15.215751
6,16.439806
7,17.225802
8,15.340650


Observations: After comparing the subset and full dataset, I observed that the mean for most values increased slightly overall.

## Task 4 - First, create a new column that include the max number of orders


In [5]:

ords_prods_merge['max_order'] = ords_prods_merge.groupby(['user_id'])['order_number'].transform(np.max)


C:\Users\casbacar\AppData\Local\Temp\ipykernel_18320\2727838813.py:1: FutureWarning: The provided callable <function max at 0x000002486117B880> is currently using SeriesGroupBy.max. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "max" instead.
  ords_prods_merge['max_order'] = ords_prods_merge.groupby(['user_id'])['order_number'].transform(np.max)


In [6]:
# Now we create the loyalty flag

ords_prods_merge.loc[ords_prods_merge['max_order'] > 40, 'loyalty_flag'] = 'Loyal customer'

In [7]:
ords_prods_merge.loc[(ords_prods_merge['max_order'] <= 40) & (ords_prods_merge['max_order'] > 10), 'loyalty_flag'] = 'Regular customer'

In [8]:
ords_prods_merge.loc[ords_prods_merge['max_order'] <= 10, 'loyalty_flag'] = 'New customer'

In [9]:
ords_prods_merge['loyalty_flag'].value_counts(dropna = False)

loyalty_flag
Regular customer    15876776
Loyal customer      10284093
New customer         6243990
Name: count, dtype: int64

## Step 5 - Create a dataframe per customer group

In [10]:
df_regular = ords_prods_merge[ords_prods_merge['loyalty_flag'] == 'Regular customer']

In [11]:
df_new = ords_prods_merge[ords_prods_merge['loyalty_flag'] == 'New customer']

In [12]:
df_loyal = ords_prods_merge[ords_prods_merge['loyalty_flag'] == 'Loyal customer']

### We check the basic stats of each customer group 


In [13]:
df_regular.groupby('loyalty_flag').agg({
    'prices': ['mean','max','min','std','sum']})

prices                                       
                       mean      max  min         std          sum
loyalty_flag                                                      
Regular customer  12.495717  99999.0  1.0  539.720919  198391693.2

In [14]:
df_new.groupby('loyalty_flag').agg({
    'prices': ['mean','max','min','std','sum']})

prices                                      
                  mean      max  min         std         sum
loyalty_flag                                                
New customer  13.29467  99999.0  1.0  597.560299  83011787.2

In [15]:
df_loyal.groupby('loyalty_flag').agg({
    'prices': ['mean','max','min','std','sum']})

prices                                       
                     mean      max  min         std          sum
loyalty_flag                                                    
Loyal customer  10.386336  99999.0  1.0  328.017787  106814042.2

### Observations:

- New customers tend to pay slightly higher prices on average (13.29), regular customers find themselves in the middle (12.50) and loyal customers have the lowest average prices (10.39). The biggest spenders are new customers.
- The price variability (std) is highest among new customers (597), followed by regulars (539) and lastly loyal customers (328).
- As per the total amount spent per group, the biggest spenders are new customers, followed by regular and loyal customers.

## Step 6 - Create a spending flag

In [16]:
# We start by creating a new column (creating flag) and calculate the mean of the prices per user

ords_prods_merge['spending_flag'] = ords_prods_merge.groupby(['user_id'])['prices'].transform(np.mean)

C:\Users\casbacar\AppData\Local\Temp\ipykernel_18320\464950357.py:1: FutureWarning: The provided callable <function mean at 0x000002486118C2C0> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  ords_prods_merge['spending_flag'] = ords_prods_merge.groupby(['user_id'])['prices'].transform(np.mean)


In [17]:
ords_prods_merge.loc[ords_prods_merge['prices'] >= 10, 'spending_flag'] = 'Big spender'

C:\Users\casbacar\AppData\Local\Temp\ipykernel_18320\3053468323.py:1: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Big spender' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  ords_prods_merge.loc[ords_prods_merge['prices'] >= 10, 'spending_flag'] = 'Big spender'


In [18]:
ords_prods_merge.loc[ords_prods_merge['prices'] <= 10, 'spending_flag'] = 'Low spender'

In [21]:
# We verify weather the spending_flag column and its values were created correctly

ords_prods_merge['spending_flag'].value_counts(dropna = False)

spending_flag
Low spender    21888779
Big spender    10516080
Name: count, dtype: int64

## Step 7 - Frequency flag

In [51]:
# We start by creating a new column (frequency_flag) and calculate the median of the the days since the last order per user

ords_prods_merge['frequency_flag'] = ords_prods_merge.groupby(['user_id'])['days_since_prior_order'].transform(np.median)

C:\Users\casbacar\AppData\Local\Temp\ipykernel_18320\3411801750.py:3: FutureWarning: The provided callable <function median at 0x00000248612F8720> is currently using SeriesGroupBy.median. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "median" instead.
  ords_prods_merge['frequency_flag'] = ords_prods_merge.groupby(['user_id'])['days_since_prior_order'].transform(np.median)


In [52]:
# Create the flags for each group using loc

ords_prods_merge.loc[ords_prods_merge['days_since_prior_order'] > 20, 'frequency_flag'] = 'Non-frequent customer'

C:\Users\casbacar\AppData\Local\Temp\ipykernel_18320\3494563258.py:3: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Non-frequent customer' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  ords_prods_merge.loc[ords_prods_merge['days_since_prior_order'] > 20, 'frequency_flag'] = 'Non-frequent customer'


In [53]:
ords_prods_merge.loc[(ords_prods_merge['days_since_prior_order'] > 10) & (ords_prods_merge['days_since_prior_order'] <= 20), 'frequency_flag'] = 'Regular customer'

In [59]:
ords_prods_merge.loc[ords_prods_merge['days_since_prior_order'] <= 10.0, 'frequency_flag'] = 'Frequent customer'

In [64]:
ords_prods_merge['frequency_flag'].value_counts(dropna = False)

frequency_flag
Frequent customer        19042466
Regular customer          6055803
Non-frequent customer     5230494
30.0                       324157
7.0                        236698
                           ...   
3.5                           854
2.5                           459
1.5                           128
0.5                            57
NaN                             5
Name: count, Length: 65, dtype: int64

Observations:

There are many N/A values in days_since_prior_order, which is why many values were excluded from the flag. The flag somehow returns numbers but they're N/As in reality.

In [66]:
ords_prods_merge.head()

,product_id,product_name,aisle_id,department_id,prices,order_id,user_id,eval_set,order_number,order_dow,...,days_since_prior_order,add_to_cart_order,reordered,_merge,Busiest days,Busiest hours,max_order,loyalty_flag,spending_flag,frequency_flag
0,1,Chocolate Sandwich Cookies,61,19,5.8,3139998,138,prior,28,6,...,3.0,5,0,both,Regularly busy,Fewer orders,32,Regular customer,Low spender,Frequent customer
1,1,Chocolate Sandwich Cookies,61,19,5.8,1977647,138,prior,30,6,...,20.0,1,1,both,Regularly busy,Most orders,32,Regular customer,Low spender,Regular customer
2,1,Chocolate Sandwich Cookies,61,19,5.8,389851,709,prior,2,0,...,6.0,20,0,both,Busiest days,Most orders,5,New customer,Low spender,Frequent customer
3,1,Chocolate Sandwich Cookies,61,19,5.8,652770,764,prior,1,3,...,NaN,10,0,both,Least busy days,Average orders,3,New customer,Low spender,9.0
4,1,Chocolate Sandwich Cookies,61,19,5.8,1813452,764,prior,3,4,...,9.0,11,1,both,Least busy days,Most orders,3,New customer,Low spender,Frequent customer


In [67]:
# Exporting to pkl file

ords_prods_merge.to_pickle('C:\\Users\\casbacar\\Documents\\Study\\Data Immersion\\Achievement 4\\03_2025_Instacart basked analysis\\03 Scripts\\Prepared Data\\ 4.8_grouping_data.pkl')